In [1]:
import os, sys, glob, argparse
import pandas as pd
import numpy as np
from tqdm import tqdm

%pylab inline

import cv2
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

Populating the interactive namespace from numpy and matplotlib


In [2]:
import nibabel as nib
from nibabel.viewers import OrthoSlicer3D

train_path = glob.glob('./中国农业大学-作物引导拍照挑战赛公开数据/train/*')
test_path = glob.glob('./中国农业大学-作物引导拍照挑战赛公开数据/test/*')

# np.random.shuffle(train_path)
# np.random.shuffle(test_path)

In [8]:
train_df = pd.read_csv('./中国农业大学-作物引导拍照挑战赛公开数据/train.csv')
train_path = './中国农业大学-作物引导拍照挑战赛公开数据/train/' + train_df['image']
train_label = train_df['label']

In [11]:
train_label.unique()

array([ 4,  7,  6,  0, 10,  3,  1,  9,  8,  2,  5])

In [39]:
class XunFeiDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = cv2.imread(self.img_path[index])            
        img = img.astype(np.float32)
        
        img /= 255.0
        img -= 1
        
        if self.transform is not None:
            img = self.transform(image = img)['image']
        img = img.transpose([2,0,1])
        return img,torch.from_numpy(np.array(self.img_label[index]))
    
    def __len__(self):
        return len(self.img_path)

In [40]:
class XunFeiNet(nn.Module):
    def __init__(self):
        super(XunFeiNet, self).__init__()
                
        model = models.resnet18(True)
        model.avgpool = nn.AdaptiveAvgPool2d(1)
        model.fc = nn.Linear(512, 11)
        self.resnet = model
        
    def forward(self, img):        
        out = self.resnet(img)
        return out

In [20]:
def train(train_loader, model, criterion, optimizer):
    model.train()
    train_loss = 0.0
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 20 == 0:
            print(loss.item())
            
        train_loss += loss.item()
    
    return train_loss/len(train_loader)
            
def validate(val_loader, model, criterion):
    model.eval()
    
    val_acc = 0.0
    
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(val_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)
            
            val_acc += (output.argmax(1) == target).sum().item()
            
    return val_acc / len(val_loader.dataset)

def predict(test_loader, model, criterion):
    model.eval()
    val_acc = 0.0
    
    test_pred = []
    with torch.no_grad():
        end = time.time()
        for i, (input, target) in enumerate(test_loader):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            test_pred.append(output.data.cpu().numpy())
            
    return np.vstack(test_pred)

In [53]:
import albumentations as A

train_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path.values[:-100], train_label.values[:-100],
            A.Compose([
            A.Resize(300, 300),
            A.RandomCrop(256, 256),
            # A.HorizontalFlip(p=0.5),
            # A.RandomContrast(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
        ])
    ), batch_size=10, shuffle=True, num_workers=1, pin_memory=False
)

val_loader = torch.utils.data.DataLoader(
    XunFeiDataset(train_path.values[-100:], train_label.values[-100:],
            A.Compose([
            A.Resize(300, 300),
            A.RandomCrop(256, 256),
            # A.HorizontalFlip(p=0.5),
            # A.RandomContrast(p=0.5),
        ])
    ), batch_size=2, shuffle=False, num_workers=1, pin_memory=False
)

test_loader = torch.utils.data.DataLoader(
    XunFeiDataset(test_path, [0] * len(test_path),
            A.Compose([
            A.Resize(300, 300),
            A.RandomCrop(256, 256),
            # A.HorizontalFlip(p=0.5),
            # A.RandomContrast(p=0.5),
        ])
    ), batch_size=2, shuffle=False, num_workers=1, pin_memory=False
)


In [47]:
model = XunFeiNet()
model = model.to('cuda')
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01)

In [49]:
for _  in range(8):
    train_loss = train(train_loader, model, criterion, optimizer)
    val_acc = validate(val_loader, model, criterion)
    
    print(train_loss, val_acc)

1.5642861127853394
2.1016221046447754
1.2280302047729492
1.3694757223129272
1.7242618799209595
1.2328886985778809
1.4003621000668098 0.86
1.19557785987854
0.9005592465400696
1.1152288913726807
1.7703393697738647
1.1053175926208496
0.9445725679397583
1.2647339665684207 0.88
1.7755447626113892
1.2936674356460571
0.8846734762191772
0.9563775062561035
0.9804204106330872
0.9901953935623169
1.1846859583566929 0.94


In [54]:
pred = None

for _ in range(10):
    if pred is None:
        pred = predict(test_loader, model, criterion)
    else:
        pred += predict(test_loader, model, criterion)

In [55]:
submit = pd.DataFrame(
    {
        'image': [x.split('/')[-1] for x in test_path],
        'label': pred.argmax(1)
})

In [61]:
submit['id'] = submit['image'].apply(lambda x: int(x.split('_')[1][:-4]))
submit = submit.sort_values(by='id')
submit[['image', 'label']].to_csv('submit2.csv', index=None)

In [62]:
submit['id']

72       1
175      2
75       3
171      4
76       5
      ... 
266    322
221    323
233    324
311    325
272    326
Name: id, Length: 326, dtype: int64